In [10]:
from random import shuffle
import glob,h5py, cv2
import numpy as np
from IPython.display import clear_output
from pandas import read_csv
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from collections import Counter
width = 128
height = 128
channels = 3
sub = 'Blake2'
filename = 'Datasets/'+sub+'_spectrogram.hdf5'
print('File to be created: ' + filename)

File to be created: Datasets/Blake2_spectrogram.hdf5


In [11]:
# load labels
trainy = '/mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/6 apr blake/labels/microDoppler/*.txt'
trainy_files = sorted(glob.glob(trainy))
print('Num. of label files: '+str(len(trainy_files)))
for i in range(0,5):
    print(str(i+1)+'. ' +trainy_files[i])

Num. of label files: 23
1. /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/6 apr blake/labels/microDoppler/11010003_1617741768_1.txt
2. /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/6 apr blake/labels/microDoppler/11010003_1617741768_2.txt
3. /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/6 apr blake/labels/microDoppler/11010003_1617741768_3.txt
4. /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/6 apr blake/labels/microDoppler/11010003_1617741768_4.txt
5. /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/6 apr blake/labels/microDoppler/11010003_1617741768_5.txt


In [12]:
# load images
trainpath = '/mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/*/microDoppler/*.png'
train_files2 = sorted(glob.glob(trainpath))
len(train_files2)
train_files2[0]
train_files = []
fname_ls = []
for i in range(0,len(trainy_files)):
#     fname = trainy_files[i][-23:].replace("txt","") + "png"
#     fname = trainy_files[i][-30:].replace("txt","png") # -23 for nonbinary labels
#     fname = fname.replace('_binary','')
    slash_idx = trainy_files[i].rfind('/')
    fname = trainy_files[i][slash_idx:].replace("txt","png")
    fname_ls.append(fname)
    idx = [j for j, s in enumerate(train_files2) if fname in s] #train_files2.index(fname) 
    if len(idx) == 1:
        train_files.append(train_files2[idx[0]])
    else:
        print('2 identical filenames at i=',str(i))
print('Num. of label files: '+str(len(train_files)))
for i in range(0,5):    
    print(str(i+1)+'. '+train_files[i])

Num. of label files: 23
1. /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/6 apr blake/microDoppler/11010003_1617741768_1.png
2. /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/6 apr blake/microDoppler/11010003_1617741768_2.png
3. /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/6 apr blake/microDoppler/11010003_1617741768_3.png
4. /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/6 apr blake/microDoppler/11010003_1617741768_4.png
5. /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/6 apr blake/microDoppler/11010003_1617741768_5.png


In [13]:
# load a single file as a numpy array
def load_file(filepath):
    dataframe = read_csv(filepath, header=None, delim_whitespace=True)
    return np.array(dataframe.values)

In [14]:
cnt = 1
images = []
winsize = 0.2 # sec
duration = 24.2
totalwin = int(duration/winsize)
for image in train_files:
    img = cv2.imread(image)
    img = cv2.resize(img, (totalwin*width, height), interpolation=cv2.INTER_CUBIC)# resize to (128,128)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # cv2 load images as BGR, convert it to RGB
    clear_output(wait=True)
    print('Loading image '+str(cnt)+'/'+str(len(train_files))+ ' '+image)
    images.append(img)  
    cnt += 1
    
images = np.swapaxes(np.array(images), 1, 2).reshape(len(trainy_files),totalwin, width, height,channels)/255.
images.shape

Loading image 23/23 /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/6 apr blake/microDoppler/11050003_1617742722_4.png


(23, 120, 128, 128, 3)

In [15]:
def pad_labels(yx):
    numpad = duration*fps - yx.shape[1]
    pads = np.repeat(yx[0,-1],numpad)
    padded = np.expand_dims(np.append(yx,pads),0)
    return padded

In [16]:
labels = []

cnt = 1
for file in trainy_files:
    clear_output(wait=True)
    print('Loading training labels #'+str(cnt))
    yx = load_file(file)
    idx = np.linspace(0,yx.shape[1]-1,totalwin).astype(int)
    labels.append(yx[0,idx])
    cnt += 1
labels = np.array(labels)   
labels = to_categorical(labels)

print(images.shape)
print(labels.shape)

Loading training labels #23
(23, 120, 128, 128, 3)
(23, 120, 19)


In [17]:
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=1)

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)
print('# of frames in a class:')
Counter(np.concatenate(np.argmax(y_train, axis=2),axis=0))

(18, 120, 128, 128, 3)
(18, 120, 19)
(5, 120, 128, 128, 3)
(5, 120, 19)
# of frames in a class:


Counter({0: 620,
         1: 478,
         2: 214,
         13: 45,
         14: 55,
         15: 52,
         3: 187,
         10: 34,
         11: 39,
         12: 35,
         4: 47,
         5: 51,
         6: 52,
         7: 58,
         8: 59,
         9: 62,
         16: 24,
         17: 25,
         18: 23})

In [18]:
f = h5py.File(filename, mode='w')
f.create_dataset("train_img", x_train.shape, np.uint8)
f.create_dataset("test_img", x_test.shape, np.uint8)  
f.create_dataset("train_labels", y_train.shape, np.uint8)
f.create_dataset("test_labels", y_test.shape, np.uint8)

f["train_img"][...] = x_train
f["test_img"][...] = x_test
f["train_labels"][...] = y_train
f["test_labels"][...] = y_test
f.close()
print(filename+' created.')

Datasets/Blake2_spectrogram.hdf5 created.
